In [1]:

# Reloading
%reload_ext autoreload
%autoreload 2

import pandas as pd
print('pandas version:{}'.format(pd.__version__))
import numpy as np
print('numpy version:{}'.format(np.__version__))
import math
from datetime import datetime
from dgpylib import dg_athena
from dgpylib import dg_s3
import os
import re
from matplotlib.ticker import MaxNLocator
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import sys
sys.path.insert(0, '../Modelling')
import helper_function2 as hf

import sys
sys.path.insert(0, '../EDA')
import data_integrity_fixer as dint
import basicstatsandplotter as bsp
import data_imputer as di
import data_integrity_fixer as dif
import parameters

##for modelling
import h2o
print('h2o version:{}'.format(h2o.__version__))
from h2o.estimators import H2OXGBoostEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators import H2OTargetEncoderEstimator
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.tree import H2OTree
from h2o.tree import H2ONode
from h2o.tree import H2OSplitNode
from h2o.tree import H2OLeafNode
from h2o.estimators.kmeans import H2OKMeansEstimator

#import xgboost as xgb
import sklearn
import category_encoders as ce
print('sklearn version:{}'.format(sklearn.__version__))
from sklearn import datasets
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import r2_score
from sklearn.preprocessing import PowerTransformer
import time
import parameters
from statsmodels.stats.outliers_influence import variance_inflation_factor

%run 'functions.ipynb'

plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 100

pandas version:1.3.4
numpy version:1.20.3
h2o version:3.36.0.4
sklearn version:1.1.2


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
cnx = dg_athena.Connect()

query = """ select * from sam_mccouig_db.prq_ptoc_em_1m_training """

df_raw = cnx.read(query)

In [5]:
conn_s3 = dg_s3.Connect('dgdatadump/DS&CA/Pricing/02 Projects/2022/Tel_Reg_2.0/Cancellation Model')
conn_s3.write(df_raw, 'churn_prq_ptoc_em_1m_training.csv')

File saved as :churn_prq_ptoc_em_1m_training.csv


In [4]:
data_list = ['df_raw3', 'df_raw6', 'df_raw9', 'df_raw12']
data_set = [df_raw3, df_raw6, df_raw9, df_raw12]

In [5]:
for i in data_set:
    print(i.cancelflag.astype(int).mean())

0.1245960040105529
0.16861956545845685
0.20206293685458737
0.2332909783989835


In [6]:
for i in data_set:
    i.drop('RouteToMarketName', axis=1, inplace=True)

In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 45 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,"2 years, 3 months and 7 days !!!"
H2O_cluster_name:,H2O_from_python_ubuntu_vwxp7y
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,27.71 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [8]:
df_filt = df_raw12.copy()

In [9]:
categorical_cols, integer_cols, float_cols, date_cols, unidentified_cols = hf.cat_num_split(df_filt)

In [10]:
predictors = list(set(categorical_cols + float_cols + integer_cols) - set(['cancelflag']))
response = 'cancelflag'

## Feature Importance
<a id='featimp'></a>

In [11]:
performance_df = pd.DataFrame(columns =['name','varimp', 'feats_stored_gbm', 'perf_metrics', 'classif_rep', 'xval_auc', 'xval_metrics'])
for idx, dataset in enumerate(data_list):
    if (dataset == 'df_raw12'):
        df = df_raw12
    elif (dataset == 'df_raw9'): 
        df = df_raw9
    elif (dataset == 'df_raw6'): 
        df = df_raw6
    else:
        df = df_raw3
    # # convert columns to factors
    # for i in df.columns:
    #     df[i] = df[i].astype('category')
        
    #train test split
    x_train, x_test, y_train, y_test = train_test_split(df[predictors], df[response], test_size = 0.2, random_state = 1)
    train = pd.concat([x_train, y_train], axis=1)
    test = pd.concat([x_test, y_test], axis=1)

    # preserve training & test data's index to be used as a key to merge prediction probability with main dataset 
    xtrain_index = x_train.index
    xtest_index = x_test.index

    # convert to h2o dataframe format
    hf_train = h2o.H2OFrame(train)
    hf_test = h2o.H2OFrame(test)

    # format target to fit the model
    hf_train['cancelflag']=hf_train['cancelflag'].asfactor()
    hf_test['cancelflag']=hf_test['cancelflag'].asfactor()

    # initialise the estimator 
    df_gbm_imp = H2OGradientBoostingEstimator(keep_cross_validation_predictions=True, nfolds = 5, seed = 1)
    start = time.time()

    # train the model
    df_gbm_imp.train(x = predictors, y = response, training_frame = hf_train)
    end = time.time()
    print(end - start)
    
    # Retrieve the variable importance
    varimp = df_gbm_imp.varimp(use_pandas=True).sort_values('scaled_importance', ascending=False)
    feats_stored_gbm = list(varimp.head(20)['variable'])
    
    #perf metrics
    perf_metrics = model_perf_stats(df_gbm_imp, hf_train, hf_test)
    
    y_true = y_test
    y_pred = df_gbm_imp.predict(hf_test)
    y_pred = y_pred.as_data_frame()
    
    # classification report
    classif_rep = classification_report(y_true, y_pred['predict'], labels=[0,1])
    
    # AUC of cross-validated holdout predictions
    xval_auc = f'AUC of cross-validated holdout predictions: {df_gbm_imp.auc(xval=True)}'
    
    xval_metrics = df_gbm_imp.cross_validation_metrics_summary()
    
    performance_df.loc[idx,:] = [dataset, varimp, feats_stored_gbm, perf_metrics, classif_rep, xval_auc, xval_metrics]

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
54.01839327812195
gbm prediction progress: |████████████████████████████████████████████████| 100%


/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/internals/blocks.py:922: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
41.54326152801514
gbm prediction progress: |████████████████████████████████████████████████| 100%


/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/internals/blocks.py:922: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
29.32128596305847
gbm prediction progress: |████████████████████████████████████████████████| 100%


/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/internals/blocks.py:922: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
18.133413076400757
gbm prediction progress: |████████████████████████████████████████████████| 100%


/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/internals/blocks.py:922: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


In [12]:
# print performance metrics for all datasets
for i in range(len(performance_df)):
    for j in ['name','perf_metrics','feats_stored_gbm','classif_rep', 'xval_auc']:

        print(performance_df.loc[i,j])

df_raw3
['MCC: 0.16007 / 0.13143', 'F1: 0.27060 / 0.26006', 'AUC: 0.64031 / 0.62245', 'AUC PR: 0.24649 / 0.21874', 'Accuracy: 0.87903 / 0.87777', 'Logloss: 0.35843 / 0.36233']
['ONS_%people_working_FT', 'ManufacturerBrandCode', 'ONS_avg_age', 'PlansAcceptedPast1YearCount', 'PlanLiveCount', 'ClientAccountDesc', 'PlansActivePast5yrCount', 'AppAge', 'ClientGroupDesc', 'PurchasePrice', 'ApplianceCode', 'Fee', 'ONS_n_families_per_population', 'ONS_avg_dependent_children_per_family', 'ONS_bedrooms per rooms', 'ONS_avg_distance_travelled_to_work(km)', 'price_diff', 'ONS_%people_working_49+', 'GoodsColour', 'ONS_avg_household_size']
              precision    recall  f1-score   support

           0       0.90      0.67      0.77     62513
           1       0.18      0.49      0.26      8899

    accuracy                           0.65     71412
   macro avg       0.54      0.58      0.51     71412
weighted avg       0.81      0.65      0.71     71412

AUC of cross-validated holdout predictio

In [13]:
# print cross-validated performance metrics for all cancellation cohorts
for i in range(len(performance_df)):
    print(performance_df.loc[i,'name'])
    print(performance_df.loc[i,'xval_metrics'])

df_raw3

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.59583294,0.03645432,0.61825204,0.5668328,0.54846543,0.6111209,0.6344935
1,auc,0.6167946,0.0018823637,0.61714303,0.6193218,0.61411124,0.6171705,0.61622626
2,aucpr,0.2175631,0.0027666995,0.21536866,0.22030607,0.21475072,0.22067074,0.21671927
3,err,0.4041671,0.03645432,0.38174796,0.43316725,0.45153457,0.38887915,0.36550647
4,err_count,23092.4,2120.4333,21844.0,24775.0,25849.0,22170.0,20824.0
5,f0point5,0.19395326,0.0055604596,0.19817168,0.19283624,0.18541278,0.19376592,0.19957972
6,f1,0.2565247,0.0033061386,0.25932455,0.25865525,0.25159964,0.2546396,0.25840455
7,f2,0.3793407,0.011950488,0.37506375,0.39268777,0.39127225,0.3712818,0.36639804
8,lift_top_group,4.394178,0.13342872,4.2387724,4.483228,4.556182,4.410821,4.281885
9,logloss,0.36246184,0.0026601504,0.3638835,0.3645213,0.3581977,0.36152044,0.36418632



See the whole table with table.as_data_frame()

df_raw6

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.54556435,0.026362866,0.51812196,0.55291533,0.51839757,0.56150675,0.5768803
1,auc,0.62143755,0.0038759457,0.617482,0.62577355,0.62360996,0.617189,0.6231332
2,aucpr,0.28136352,0.0047666025,0.2762652,0.2878656,0.28425398,0.28080338,0.27762944
3,err,0.45443562,0.026362866,0.48187804,0.4470847,0.48160243,0.43849325,0.42311972
4,err_count,19674.0,1212.9845,20927.0,19323.0,20942.0,18928.0,18250.0
5,f0point5,0.24878915,0.0031881437,0.24509513,0.2533339,0.24792984,0.24711743,0.25046945
6,f1,0.3225845,0.0037912936,0.32189494,0.32707644,0.3258,0.3181065,0.3200447
7,f2,0.45892206,0.013867518,0.46878836,0.46137825,0.47498357,0.44632027,0.44313985
8,lift_top_group,4.0142903,0.09080743,3.9272923,3.9699244,3.9574683,4.1438193,4.072946
9,logloss,0.43721113,0.0024863705,0.43882143,0.43951446,0.43865243,0.43498933,0.43407807



See the whole table with table.as_data_frame()

df_raw9

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.51351833,0.007421214,0.5166009,0.51459485,0.52092594,0.5011067,0.5143634
1,auc,0.61954945,0.0044063814,0.6180751,0.6124592,0.6223943,0.6230286,0.62179035
2,aucpr,0.3224064,0.0067246174,0.32744983,0.31265894,0.3298276,0.32196087,0.32013485
3,err,0.48648164,0.007421214,0.4833991,0.48540512,0.47907406,0.49889335,0.48563656
4,err_count,13061.8,140.33069,12987.0,13054.0,12935.0,13299.0,13034.0
5,f0point5,0.28401414,0.006326591,0.2837141,0.27500117,0.29266182,0.28297287,0.28572077
6,f1,0.36428753,0.0074720015,0.36272633,0.3529295,0.37345603,0.365536,0.36678973
7,f2,0.5078612,0.010160475,0.502734,0.49248818,0.51587087,0.51612645,0.51208663
8,lift_top_group,3.7247496,0.102922365,3.640082,3.8126466,3.8578863,3.6716404,3.641493
9,logloss,0.4846656,0.0034845513,0.48565364,0.4797979,0.4894926,0.48391876,0.4844651



See the whole table with table.as_data_frame()

df_raw12

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.54466593,0.030313266,0.5025636,0.5578078,0.5759394,0.52415025,0.56286865
1,auc,0.6343176,0.0070356927,0.63346535,0.6224008,0.6377931,0.6390045,0.6389242
2,aucpr,0.37504265,0.010439755,0.3785877,0.3595672,0.3706101,0.37932688,0.38712135
3,err,0.45533404,0.030313266,0.49743637,0.4421922,0.42406055,0.47584972,0.43713135
4,err_count,4872.8,313.4752,5336.0,4712.0,4593.0,5054.0,4669.0
5,f0point5,0.33268633,0.00870086,0.32171017,0.33100685,0.34040672,0.3278143,0.3424936
6,f1,0.4114783,0.0055142087,0.40763766,0.40490022,0.41318512,0.4124622,0.41920638
7,f2,0.5398508,0.016427407,0.5561951,0.52126694,0.525546,0.55604315,0.5402026
8,lift_top_group,3.4940062,0.21892786,3.6963341,3.4050872,3.1557593,3.6463475,3.566502
9,logloss,0.5189237,0.0025382421,0.51807153,0.5220884,0.51896954,0.51527005,0.520219



See the whole table with table.as_data_frame()



In [14]:
# print feat imp for 12 month journey
print(performance_df.loc[3,'varimp'])

                                  variable  relative_importance  \
0                        ClientAccountDesc           311.499634   
1                                   AppAge           276.248657   
2              PlansAcceptedPast1YearCount           274.688507   
3                            PlanLiveCount           257.863342   
4                    ManufacturerBrandCode           241.274979   
5                   ONS_%people_working_FT           232.545837   
6                         ONS_avg_bedrooms           190.392044   
7              PlansAcceptedPast5YearCount           172.148300   
8                   ONS_population_density           142.732910   
9                            ApplianceCode           129.561783   
10                         ClientGroupDesc           121.483597   
11                           ONS_avg_rooms           116.259758   
12                             ONS_avg_age           104.791580   
13                        HPI_AveragePrice           100.68338

In [15]:
# save performance metric to pickle
pd.to_pickle(performance_df, "feat_imp_performance_df.pkl")